In [1]:
%pylab inline
import numpy as np
import scipy as sc
import pandas as pd

import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("talk")
#rc('axes', labelsize=20, titlesize=20)

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import scipy.stats as ss

import timeit
import pickle

from ABC_PMC import ABC_sample, ABC_PMC

Populating the interactive namespace from numpy and matplotlib


:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
######
# set up for the normal ABC example
######

prior_mean = -4.0
prior_sd = 3
likelihood_sd = 1
original_data_size = 100

def NormalPriorFunction(x):
    return ss.norm.pdf(x=x,loc=prior_mean, scale=prior_sd)

def NormalPriorSampler(n):
    return np.random.normal(loc=prior_mean, scale=prior_sd, size=n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return np.random.normal(loc=param, scale=likelihood_sd, size=n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(loc=0,scale=likelihood_sd,size=original_data_size)

In [18]:
k = 20
requested_sample_size = np.linspace(start=50,stop=1000,num=20) #n=50,100,...,1000
tolerance_seq = np.linspace(start=1, stop= 0.01, num=4) #T = 4

In [10]:
ABC_benchmark = []

for sample_size in requested_sample_size:
    print("ABC - ",float(len(ABC_benchmark))/len(requested_sample_size)*100,"%")
    for rep in range(k):       
        #Run ABC
        start_time = timeit.default_timer()
        ABC_run = ABC_sample(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq[-1], data , sample_size)
        run_time_ABC = timeit.default_timer() - start_time
        niter_ABC = ABC_run[1]
        times_for_iteration_ABC = np.mean(ABC_run[2])
        output_dict = {'k': rep, 'sample_size': sample_size, 'run_time': run_time_ABC, 'niter':niter_ABC, 'times_1000_iter': times_for_iteration_ABC}
        ABC_benchmark.append(output_dict)
    

file_name = "data/New_ABC_benchmark_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(requested_sample_size))+".p"
pickle.dump(ABC_benchmark, open( file_name, "wb" ) ) #save result in a file
#You can load using:
#test = pickle.load( open( "data/ABC_benchmark_k_2_tol_001.p", "rb" ) )

#print(pd.DataFrame(ABC_benchmark))
print("DONE :D")



('ABC - ', 0.0, '%')
('ABC - ', 100.0, '%')
DONE :D


In [13]:
PMC_benchmark = []

for sample_size in requested_sample_size:
    print("PMC 1 - ",float(len(PMC_benchmark))/len(requested_sample_size)*100,"%")
    for rep in range(k):
        #Run ABC
        start_time = timeit.default_timer()
        PMC_run = ABC_PMC(NormalPriorFunction,NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq, data , sample_size)
        run_time_PMC = timeit.default_timer() - start_time
        niter_PMC = PMC_run[1]
        times_for_iteration_PMC = np.mean(PMC_run[2])
        output_dict = {'k': rep, 'sample_size': sample_size, 'run_time': run_time_PMC, 'niter':niter_PMC, 'times_1000_iter': times_for_iteration_PMC}
        PMC_benchmark.append(output_dict)

    
file_name = "data/New_PMC_benchmark_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(requested_sample_size))+"_T_"+str(len(tolerance_seq))+".p"
pickle.dump(PMC_benchmark, open( file_name, "wb" ) ) #save result in a file
#You can load using:
#test = pickle.load( open( "data/ABC_benchmark_k_2_tol_001.p", "rb" ) )

#print(pd.DataFrame(PMC_benchmark))
print("PMC - 100%")

('PMC 1 - ', 0.0, '%')
('PMC 1 - ', 100.0, '%')
PMC - 100%


/usr/lib64/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [16]:
#PMC benchmark for different values of T but fixed n
sample_size_seq = [100,250,500,1000]

PMC_benchmark2 = []
for sample_size in sample_size_seq:
    print("PMC 2 - ",float(sample_size_seq.index(sample_size))/len(sample_size_seq)*100,"%")
    for T in range(2,11):
        tolerance_seq = np.linspace(start=1, stop= 0.01, num=T)
        for rep in range(k):
            #Run ABC
            start_time = timeit.default_timer()
            PMC_run = ABC_PMC(NormalPriorFunction,NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, tolerance_seq, data , sample_size)
            run_time_PMC = timeit.default_timer() - start_time
            niter_PMC = PMC_run[1]
            times_for_iteration_PMC = np.mean(PMC_run[2])
            output_dict = {'k': rep, 'T':T, 'sample_size': sample_size, 'run_time': run_time_PMC, 'niter':niter_PMC, 'times_1000_iter': times_for_iteration_PMC}
            PMC_benchmark2.append(output_dict)
        

file_name = "data/New_PMC_benchmark2_k_"+str(k)+"_tol_"+str(tolerance_seq[-1])+"_num_"+str(len(sample_size_seq))+"_T_2_10.p"
pickle.dump(PMC_benchmark2, open( file_name, "wb" ) ) #save result in a file
#print(pd.DataFrame(PMC_benchmark2))
print("PMC 2 - 100%")

('PMC 2 - ', 0.0, '%')
('PMC 2 - ', 25.0, '%')
('PMC 2 - ', 50.0, '%')
('PMC 2 - ', 75.0, '%')
PMC 2 - 100%


In [17]:
print(PMC_benchmark2)

[{'niter': 11397, 'k': 0, 'times_1000_iter': 0.052038598060607913, 'sample_size': 100, 'T': 2, 'run_time': 0.5693659782409668}, {'niter': 14517, 'k': 1, 'times_1000_iter': 0.051969190438588463, 'sample_size': 100, 'T': 2, 'run_time': 0.7160661220550537}, {'niter': 7371, 'k': 0, 'times_1000_iter': 0.052959958712259926, 'sample_size': 100, 'T': 3, 'run_time': 0.36411499977111816}, {'niter': 8031, 'k': 1, 'times_1000_iter': 0.053313561848231723, 'sample_size': 100, 'T': 3, 'run_time': 0.39335203170776367}, {'niter': 5815, 'k': 0, 'times_1000_iter': 0.049627224604288735, 'sample_size': 100, 'T': 4, 'run_time': 0.29714393615722656}, {'niter': 6828, 'k': 1, 'times_1000_iter': 0.056638574600219725, 'sample_size': 100, 'T': 4, 'run_time': 0.34854602813720703}, {'niter': 6028, 'k': 0, 'times_1000_iter': 0.056738615036010742, 'sample_size': 100, 'T': 5, 'run_time': 0.3261430263519287}, {'niter': 5857, 'k': 1, 'times_1000_iter': 0.06154102087020874, 'sample_size': 100, 'T': 5, 'run_time': 0.30990

In [ ]:
print("DONE! :D")